# Refer to 
https://github.com/Azure/ViennaDocs/blob/master/PrivatePreview/notebooks/01.train-within-notebook.ipynb
https://github.com/Azure/ViennaDocs/blob/master/PrivatePreview/notebooks/11.production-deploy-to-aks.ipynb

In [1]:
from azureml.core import Workspace
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.image import Image
from azureml.core.model import Model

In [2]:
import azureml.core
print(azureml.core.VERSION)

0.1.4


In [3]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

Found the config file in: /home/jayaubuntudsvm/Desktop/AKSDeploymentTutorial/Keras_Tensorflow/aml_config/config.json
jayavienna
jayavienna
eastus2
edf507a2-6235-46c5-b560-fd463ba2e771


In [4]:
#Creating the model pickle file
import tensorflow as tf
from resnet152 import ResNet152
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input, decode_predictions

model = ResNet152(weights='imagenet')

Using TensorFlow backend.


In [5]:
model

In [6]:
model_json = model.to_json()
with open("model_resnet.json", "w") as json_file:
    json_file.write(model_json)

In [7]:
model.save_weights("model_resnet_weights.h5")

In [8]:
#from keras.models import model_from_json
#model = model_from_json(model_json)

In [9]:
model.load_weights('model_resnet_weights.h5')

In [10]:
model

In [11]:
#Register the model
from azureml.core.model import Model
model = Model.register(model_path = "model_resnet_weights.h5", # this points to a local file
                       model_name = "resnet_model", # this is the name the model is registered as
                       tags = ["dl", "resnet"],
                       description = "resnet 152 model",
                       workspace = ws)

print(model.name, model.description, model.version)

Registering model resnet_model
resnet_model resnet 152 model 3


In [12]:
#define init() function
def init():
    import tensorflow as tf
    from resnet152 import ResNet152
    from keras.preprocessing import image
    from keras.applications.imagenet_utils import preprocess_input, decode_predictions

    import numpy as np
    import timeit as t
    import base64
    import json
    from PIL import Image, ImageOps
    from io import BytesIO
    import logging

    global model
    model = ResNet152(weights='imagenet')
    print('Model loaded')

In [13]:
init()

Model loaded


In [14]:
#define run() function 
def run(img_path):
    
    import tensorflow as tf
    from resnet152 import ResNet152
    from keras.preprocessing import image
    from keras.applications.imagenet_utils import preprocess_input, decode_predictions

    import numpy as np
    import timeit as t
    import base64
    import json
    from PIL import Image, ImageOps
    from io import BytesIO
    import logging   
    
    model = ResNet152(weights='imagenet')
    print('Model loaded')
  
    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    
    preds = model.predict(img)
    print('Predicted:', decode_predictions(preds, top=3))

In [15]:
import wget
wget.download('https://upload.wikimedia.org/wikipedia/commons/thumb/6/68/Lynx_lynx_poing.jpg/220px-Lynx_lynx_poing.jpg')

'220px-Lynx_lynx_poing.jpg'

In [16]:
img_path = '220px-Lynx_lynx_poing.jpg'

In [17]:
run(img_path)

Model loaded
Predicted: [[('n02127052', 'lynx', 0.9959085), ('n02128385', 'leopard', 0.0011504436), ('n02123159', 'tiger_cat', 0.0009417995)]]
